In [1]:
# Imports
import json, os, re, shutil, sys, time
import seaborn as sns
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML
from collections import defaultdict

# NLTK for NLP utils and corpora
import nltk
from nltk.corpus import treebank
from nltk.text import Text
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

# Import spacy
import spacy

import pickle

import sklearn
from sklearn.model_selection import train_test_split

# NumPy, Pandas and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf

from numpy.random import seed
from pandas import read_csv, DataFrame
from sklearn.preprocessing import minmax_scale

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.embeddings import Embedding
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Activation, Dropout, Conv1D, MaxPooling1D, Bidirectional, Flatten, TimeDistributed

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from datetime import datetime


Using TensorFlow backend.


## AEG Long Essay Sentence Level Classification

In [2]:
# Read the data
aeg_long = pd.read_csv("../data-DNC/AEG/training_set_rel3.tsv",sep='\t',encoding = "latin1")
aeg_long.shape

(12976, 28)

In [4]:
# Split the data into train and test. We need to do this first to ensure that when we split to 
# sentence level, we have sentences of a given essay in either training or test but not on both.

train_comb,test_comb = train_test_split(aeg_long,test_size=0.2, random_state=42)

In [5]:
# Create train and test data frames with relevant fields

x_train_df = train_comb.filter(['essay_id','essay_set','essay','domain1_score'], axis=1)
x_test_df = test_comb.filter(['essay_id','essay_set','essay','domain1_score'], axis=1)

x_train_df = x_train_df.reset_index()
x_test_df = x_test_df.reset_index()
print(x_train_df.shape)
print(x_test_df.shape)
print(x_train_df.essay_id[:5].values)
print(x_test_df.essay_id[:5].values)

(10380, 5)
(2596, 5)
[ 3567  4233  6518 15174 18855]
[ 9908  9872   305 12771  6839]


#### Split essay into sentences

In [7]:
# We will use spacy to split the essay into sentences. 
# Load spacy large english module
nlp = spacy.load('en_core_web_lg')

In [14]:
# Define a function to split essay into sentences
def create_sentences_df(df):
    """ Function to split essay data into individual sentences. Returns a dataframe"""
    start=datetime.now()
    aeg_long_sentence = pd.DataFrame(columns=['essay_id','essay_set','sentence','domain1_score'])
    for i in range(len(df)):
        if i%1000 == 0:
            print("At iteration :",i)
            print("Duration: ",datetime.now()-start)
        sentence = nlp(df.essay[i])
        for s in sentence.sents:
            aeg_long_sentence = aeg_long_sentence.append({'essay_id' : df.essay_id[i],
                                                          'essay_set' : df.essay_set[i],'sentence' : s.text, 
                                                          'domain1_score' : df.domain1_score[i]},
                                                         ignore_index=True)
    return aeg_long_sentence

In [15]:
# Split train data into sentences
x_train_sentence_df = create_sentences_df(x_train_df)

At iteration : 0
Duration:  0:00:00.001094
At iteration : 1000
Duration:  0:01:13.910230
At iteration : 2000
Duration:  0:02:41.471731
At iteration : 3000
Duration:  0:04:57.690647
At iteration : 4000
Duration:  0:07:51.580886
At iteration : 5000
Duration:  0:11:39.480142
At iteration : 6000
Duration:  0:16:11.116163
At iteration : 7000
Duration:  0:21:46.361166
At iteration : 8000
Duration:  0:27:57.529110
At iteration : 9000
Duration:  0:34:42.670452
At iteration : 10000
Duration:  0:42:12.653912


In [17]:
# Split test data into sentences
x_test_sentence_df = create_sentences_df(x_test_df)

At iteration : 0
Duration:  0:00:00.001098
At iteration : 1000
Duration:  0:01:15.384433
At iteration : 2000
Duration:  0:02:53.557499


#### Normalize the scores

In [19]:
# Each essay set has a different scoring range. We need to normalize the scores to a standard scale for training.
def normalize_score(essay):
    """ Normalizes the domain score based on percentage"""
    score = 0
    score = float(essay[3])
    essay_set = essay[1]
    if essay_set == 1:
        div = 12
    elif essay_set == 2:
        div = 5
    elif essay_set == 3:
        div = 3
    elif essay_set == 4:
        div = 3
    elif essay_set == 5:
        div = 4
    elif essay_set == 6:
        div = 4
    elif essay_set == 7:
        div = 25
    elif essay_set == 8:
        div = 50
    return score/div

In [20]:
x_train_sentence_df['Norm_Score'] = x_train_sentence_df.apply(normalize_score,axis=1)
x_test_sentence_df['Norm_Score'] = x_test_sentence_df.apply(normalize_score,axis=1)

In [25]:
x_train_sentence_df.head(5)

,essay_id,essay_set,sentence,domain1_score,Norm_Score
0,4075,2,Did your child ever bring home a book?,3.0,0.6
1,4075,2,"A piece of music, movie or magazine?",3.0,0.6
2,4075,2,Did you ever stop to think that the piece of i...,3.0,0.6
3,4075,2,There are points in time where there isn't an ...,3.0,0.6
4,4075,2,The driving question is do you want to remove ...,3.0,0.6


In [61]:
# Store this dataset in pickle format so that we don't have to redo the above steps.
x_train_sentence_df.to_pickle("./x_train_sentence_df.pkl")
x_test_sentence_df.to_pickle("./x_test_sentence_df.pkl")

In [8]:
# Open the pickled version
x_train_sentence_df = pd.read_pickle("/home/pkurapati/x_train_sentence_df.pkl")
x_test_sentence_df = pd.read_pickle("/home/pkurapati/x_test_sentence_df.pkl")
print(x_train_sentence_df.shape)
print(x_test_sentence_df.shape)

(147123, 5)
(36778, 5)


In [9]:
def append_sentence(sid,sentence):
    sentence = sid_dict[sid] + sentence
    return sentence

In [10]:
sid_dict = {1:"One. ",2:"Two. ",3:"Three. ",4:"Four. ",5:"Five ",6:"Six. ",7:"Seven. ",8:"Eight. "}

In [11]:
x_train_sentence_df['mod_sentence'] = np.vectorize(append_sentence)(x_train_sentence_df['essay_set'],
                                                                    x_train_sentence_df['sentence'])

x_test_sentence_df['mod_sentence'] = np.vectorize(append_sentence)(x_test_sentence_df['essay_set'],
                                                                    x_test_sentence_df['sentence'])

In [12]:
x_train_sentence_df.head(5)

,essay_id,essay_set,sentence,domain1_score,Norm_Score,mod_sentence
0,3567,2,There are many types of reading materials for ...,4,0.8,Two. There are many types of reading materials...
1,3567,2,"You can find things on cars, trucks, sports, a...",4,0.8,"Two. You can find things on cars, trucks, spor..."
2,3567,2,There are some materials in a library though t...,4,0.8,Two. There are some materials in a library tho...
3,3567,2,But should those materials be removed from the...,4,0.8,Two. But should those materials be removed fro...
4,3567,2,Some think that they should and others think t...,4,0.8,Two. Some think that they should and others th...


In [13]:
# Create train and test text and labels

x_train = x_train_sentence_df['mod_sentence'].values
y_train = x_train_sentence_df['Norm_Score'].values
x_test = x_test_sentence_df['mod_sentence'].values
y_test = x_test_sentence_df['Norm_Score'].values
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(147123,)
(147123,)
(36778,)
(36778,)


In [14]:
vocabulary_size = 50000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(x_train)
train_seq = tokenizer.texts_to_sequences(x_train)
train_data = pad_sequences(train_seq)

In [15]:
max_len_class = train_data.shape[1]
max_words_class = vocabulary_size

In [16]:
test_seq = tokenizer.texts_to_sequences(x_test)
test_data = pad_sequences(test_seq, maxlen=max_len_class)
test_data.shape

(36778, 176)

In [17]:
embeddings_index = dict()
f = open('/home/pkurapati/W266-NLP-Project/data-DNC/glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1917494 word vectors.


In [18]:
embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [19]:
def RMSE(actual, predict):
    diff = actual - predict
    diff = sum(diff**2) / len(actual)
    return np.sqrt(diff)

In [33]:
#model definitions

def FF_NN():
    """ Simple feed forward NN"""
    model_ff = Sequential()
    model_ff.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_ff.add(tf.keras.layers.Flatten())
    model_ff.add(tf.keras.layers.Dense(50,activation='tanh'))
    model_ff.add(tf.keras.layers.Dropout(0.1))
    model_ff.add(tf.keras.layers.Dense(1,activation='sigmoid'))
    model_ff.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_ff

def GRU():
    """ Gated Recurrent Unit"""
    model_gru = Sequential()
    model_gru.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_gru.add(tf.keras.layers.GRU(32,activation='tanh'))
    model_gru.add(tf.keras.layers.Dropout(0.1))
    model_gru.add(tf.keras.layers.Dense(1,name='out_layer'))
    model_gru.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_gru

def CNN_FF():
    """ CNN with Feed Forward NN """
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
    model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=4))
    model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    return model_conv

def CNN_lstm():
    """ CNN with single layer LSTM & Feed Forward NN"""
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
    model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=4))
    model_conv.add(tf.keras.layers.LSTM(100))
    model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    return model_conv

def stack_lstm():
    """ Three layered stacked LSTM."""
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_conv.add(tf.keras.layers.LSTM(32,return_sequences=True))
    model_conv.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model_conv.add(tf.keras.layers.Dropout(0.2))
    model_conv.add(tf.keras.layers.LSTM(32))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    return model_conv

def stateful_stacked_lstm():
    # In stateful, total samples needs to be divisible by batch size
    # we have 147026 samples, so selecting 6683 (6683*22=147026)
    # The test sample need to be a multiple of 6683 as well
    batch_size=2
    model_conv = Sequential()
    # In stateful, we have to pass batch_input_shape to the first layer
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], 
                                             trainable=False,batch_input_shape=(batch_size,max_len_class)))
    model_conv.add(tf.keras.layers.LSTM(32,stateful=True,return_sequences=True))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.LSTM(32))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    #sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    #model_conv.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_conv


#### Common Functions

In [21]:
# Define function to get the multiplication factor. We can get it back from the essay set, but
# it is better to do it from the score, because there are scores with value 0, and its MF should be 0

def find_mult_factor(x):
    """ Function to find the multiplication factor for denormalizing"""
    if x[1] == 0:
        return 0
    else:
        return np.around(x[0]/x[1])

In [22]:
def denormalize(x):
    """ Function to Denormalize the score"""
    return np.around(x[2] * x[3])

In [23]:
def find_max_min_mean_score(df):
    """ Function to find the max, min and rounded mean of sentence scores"""
    new_df = pd.DataFrame(columns=['essay_id','essay_set','Orig_Score','Max_Score','Min_Score','Mean_Score'])
    essay_ids = np.unique(df.essay_id)
    for e_id in essay_ids:
        df_temp = df[df.essay_id == e_id]
        max_score = np.max(df_temp.pred_score)
        min_score = np.min(df_temp.pred_score)
        # we need to round the mean so that kappa score doesnt complain
        mean_score = np.around(np.mean(df_temp.pred_score))
        new_df = new_df.append({'essay_id':e_id,'essay_set':int(np.unique(df_temp.essay_set)),
                                'Orig_Score':int(np.unique(df_temp.orig_score)),'Max_Score':max_score,
                                'Min_Score':min_score,'Mean_Score':mean_score},ignore_index=True)
    return new_df

In [24]:
# To calculate the per essay set scores

def essay_set_metrics(df):
    """ Calculate per essay set metrics"""
    set_df = pd.DataFrame(columns=['essay_set','RMSE','Kappa','Accuracy'])
    e_sets = np.unique(df.essay_set)
    for e_s in e_sets:
        df_s = df[df.essay_set == e_s]
        original_score = df_s.Orig_Score.values.astype(int)
        predicted_score = df_s.Mean_Score.values.astype(int)
        rmse = RMSE(original_score,predicted_score)
        kappa = cohen_kappa_score(original_score,predicted_score)
        accuracy = accuracy_score(original_score,predicted_score)
        set_df = set_df.append({'essay_set':e_s,'RMSE':rmse,'Kappa':kappa,'Accuracy':accuracy},
                              ignore_index=True)
    return set_df

In [25]:
def sentence_count(df):
    """ Returns the number of sentences in an essay """
    essay_count = df.groupby('essay_id').count()
    essay_count = essay_count.drop(['sentence','domain1_score','Norm_Score'],axis=1)
    essay_count.columns = ['Number_of_Sentences']
    return essay_count

#### CNN_LSTM

In [26]:
# Train & Predict
estimator = KerasRegressor(build_fn=CNN_lstm, epochs=20, batch_size=500)
estimator.fit(train_data, y_train)
prediction_cnn_glove=estimator.predict(test_data)
# Metric before denormalizing. This is not very useful as the scale per essay set is different
rmse_val = RMSE(y_test,prediction_cnn_glove)
print("RMSE before denormalizing: ",rmse_val)

Epoch 1/20
147123/147123 [==============================] - 45s 307us/step - loss: 0.0345 - acc: 0.0976
Epoch 2/20
147123/147123 [==============================] - 27s 186us/step - loss: 0.0273 - acc: 0.0997
Epoch 3/20
147123/147123 [==============================] - 27s 184us/step - loss: 0.0262 - acc: 0.1006
Epoch 4/20
147123/147123 [==============================] - 27s 186us/step - loss: 0.0255 - acc: 0.1012
Epoch 5/20
147123/147123 [==============================] - 28s 190us/step - loss: 0.0247 - acc: 0.1016
Epoch 6/20
147123/147123 [==============================] - 27s 185us/step - loss: 0.0240 - acc: 0.1021
Epoch 7/20
147123/147123 [==============================] - 27s 184us/step - loss: 0.0234 - acc: 0.1025
Epoch 8/20
147123/147123 [==============================] - 28s 187us/step - loss: 0.0227 - acc: 0.1028
Epoch 9/20
147123/147123 [==============================] - 27s 185us/step - loss: 0.0221 - acc: 0.1031
Epoch 10/20
147123/147123 [==============================] - 27s

In [27]:
# Denormalize

# Construct pandas data frame of scores of each sentences
cnn_df = pd.DataFrame([x_test_sentence_df['domain1_score'].astype(np.double), y_test.astype(np.double),
                      prediction_cnn_glove.astype(np.double)]).transpose()
cnn_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
# Find Multiplication factor using the function we defined before
cnn_df['Mult_Factor'] = cnn_df.apply(find_mult_factor,axis=1)
cnn_df['Denorm_Pred_Score'] = cnn_df.apply(denormalize,axis=1)

# Extract the scores of each sentences
orig_score = cnn_df.Orig_Score.as_matrix(columns=None)
orig_score = orig_score.astype(np.int)
pred_score = cnn_df.Denorm_Pred_Score.as_matrix(columns=None)
pred_score = pred_score.astype(np.int)
rmse_cnn = RMSE(orig_score,pred_score)

# Provide the metrics at sentence level
print("RMSE: ",rmse_cnn)
cohen_kappa = cohen_kappa_score(orig_score,pred_score)
print("Cohen Kappa: ",cohen_kappa)
accuracy = accuracy_score(orig_score,pred_score)
print("Accuracy: ",accuracy)

RMSE:  2.6789887651390716
Cohen Kappa:  0.26229815533140743
Accuracy:  0.32359018978737286


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [28]:
# We will combine the sentences back to essay in this section
result_df = pd.DataFrame([x_test_sentence_df.essay_id.values,x_test_sentence_df.essay_set.values,
                         x_test_sentence_df.sentence.values,cnn_df.Orig_Score.values,
                          cnn_df.Denorm_Pred_Score.values]).transpose()
result_df.columns = ['essay_id','essay_set','sentence','orig_score','pred_score']
cnn_test_df = find_max_min_mean_score(result_df)

#Convert the dataframe parameters into integers as they are returned as floats
cnn_test_df.essay_id = cnn_test_df.essay_id.astype(int)
cnn_test_df.essay_set = cnn_test_df.essay_set.astype(int)
cnn_test_df.Orig_Score = cnn_test_df.Orig_Score.astype(int)
cnn_test_df.Max_Score = cnn_test_df.Max_Score.astype(int)
cnn_test_df.Min_Score = cnn_test_df.Min_Score.astype(int)
cnn_test_df.Mean_Score = cnn_test_df.Mean_Score.astype(int)

# Extract the scores
orig_score = cnn_test_df.Orig_Score.values
max_pred_score = cnn_test_df.Max_Score.values
min_pred_score = cnn_test_df.Min_Score.values
mean_pred_score = cnn_test_df.Mean_Score.values

# Compare the metrics if Max score of all sentences are taken
rmse_max_cnn = RMSE(orig_score,max_pred_score)
cohen_kappa_max_cnn = cohen_kappa_score(orig_score,max_pred_score)
accuracy_max_cnn = accuracy_score(orig_score,max_pred_score)

# Compare the metrics if Min score of all sentences are taken
rmse_min_cnn = RMSE(orig_score,min_pred_score)
cohen_kappa_min_cnn = cohen_kappa_score(orig_score,min_pred_score)
accuracy_min_cnn = accuracy_score(orig_score,min_pred_score)

# Compare the metrics if Rounded Mean score of all sentences are taken
rmse_mean_cnn = RMSE(orig_score,mean_pred_score)
cohen_kappa_mean_cnn = cohen_kappa_score(orig_score,mean_pred_score)
accuracy_mean_cnn = accuracy_score(orig_score,mean_pred_score)

print("### CNN_LSTM Results : MAX Score of all Sentences ###")
print("RMSE: ",rmse_max_cnn)
print("Cohen Kappa: ",cohen_kappa_max_cnn)
print("Accuracy: ",accuracy_max_cnn)

print("### CNN_LSTM Results : Min Score of all Sentences ###")
print("RMSE: ",rmse_min_cnn)
print("Cohen Kappa: ",cohen_kappa_min_cnn)
print("Accuracy: ",accuracy_min_cnn)

print("### CNN_LSTM Results : Mean Score of all Sentences ###")
print("RMSE: ",rmse_mean_cnn)
print("Cohen Kappa: ",cohen_kappa_mean_cnn)
print("Accuracy: ",accuracy_mean_cnn)

# Provide per-essay-set metrics based on mean score
essay_set_results = essay_set_metrics(cnn_test_df)
essay_set_results

### CNN_LSTM Results : MAX Score of all Sentences ###
RMSE:  2.656720988335306
Cohen Kappa:  0.2583493844094005
Accuracy:  0.3474576271186441
### CNN_LSTM Results : Min Score of all Sentences ###
RMSE:  2.6370011449600854
Cohen Kappa:  0.24143335205972272
Accuracy:  0.3332049306625578
### CNN_LSTM Results : Mean Score of all Sentences ###
RMSE:  1.9841436152603542
Cohen Kappa:  0.3371955856765654
Accuracy:  0.42180277349768874


,essay_set,RMSE,Kappa,Accuracy
0,1.0,1.407435,0.043304,0.262295
1,2.0,0.763471,0.265039,0.566845
2,3.0,0.777683,0.100836,0.422156
3,4.0,0.612372,0.521764,0.666667
4,5.0,0.783896,0.245669,0.507246
5,6.0,0.727393,0.236521,0.534392
6,7.0,3.858851,-0.000656,0.079470
7,8.0,5.424330,-0.011183,0.036496


#### Stacked LSTM

In [29]:
# Train the model
estimator_lstm = KerasRegressor(build_fn=stack_lstm, epochs=20, batch_size=500)
estimator_lstm.fit(train_data, y_train)

# Predict for test data
prediction_lstm=estimator_lstm.predict(test_data)
rmse_val = RMSE(y_test,prediction_lstm)
# Find the overall RMSE value. This is not very relevant considering different scale for each set.
print("RMSE: ",rmse_val)

# Construct pandas data frame of scores of each sentences
lstm_df = pd.DataFrame([x_test_sentence_df['domain1_score'].astype(np.double), y_test.astype(np.double),
                      prediction_lstm.astype(np.double)]).transpose()
lstm_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
# Find Multiplication factor using the function we defined before
lstm_df['Mult_Factor'] = lstm_df.apply(find_mult_factor,axis=1)
# Find the denormalized predicted score
lstm_df['Denorm_Pred_Score'] = lstm_df.apply(denormalize,axis=1)

# Extract the scores of each sentences
orig_score = lstm_df.Orig_Score.as_matrix(columns=None)
orig_score = orig_score.astype(np.int)
pred_score = lstm_df.Denorm_Pred_Score.as_matrix(columns=None)
pred_score = pred_score.astype(np.int)

# Provide the metrics at sentence level
rmse_lstm = RMSE(orig_score,pred_score)
cohen_kappa = cohen_kappa_score(orig_score,pred_score)
accuracy = accuracy_score(orig_score,pred_score)
print("RMSE Sentence Level: ",rmse_lstm)
print("Kappa Sentence Level: ",cohen_kappa)
print("Accuracy Sentence Level: ",accuracy)

Epoch 1/20
147123/147123 [==============================] - 317s 2ms/step - loss: 0.0486 - acc: 0.0937
Epoch 2/20
147123/147123 [==============================] - 312s 2ms/step - loss: 0.0305 - acc: 0.0987
Epoch 3/20
147123/147123 [==============================] - 313s 2ms/step - loss: 0.0279 - acc: 0.0992
Epoch 4/20
147123/147123 [==============================] - 312s 2ms/step - loss: 0.0269 - acc: 0.1002
Epoch 5/20
147123/147123 [==============================] - 314s 2ms/step - loss: 0.0264 - acc: 0.1005
Epoch 6/20
147123/147123 [==============================] - 312s 2ms/step - loss: 0.0261 - acc: 0.1007
Epoch 7/20
147123/147123 [==============================] - 315s 2ms/step - loss: 0.0259 - acc: 0.1009
Epoch 8/20
147123/147123 [==============================] - 313s 2ms/step - loss: 0.0256 - acc: 0.1011
Epoch 9/20
147123/147123 [==============================] - 313s 2ms/step - loss: 0.0253 - acc: 0.1014
Epoch 10/20
147123/147123 [==============================] - 314s 2ms/ste

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [30]:
# We will combine the sentences back to essay in this section

result_df = pd.DataFrame([x_test_sentence_df.essay_id.values,x_test_sentence_df.essay_set.values,
                         x_test_sentence_df.sentence.values,lstm_df.Orig_Score.values,
                          lstm_df.Denorm_Pred_Score.values]).transpose()
result_df.columns = ['essay_id','essay_set','sentence','orig_score','pred_score']
lstm_test_df = find_max_min_mean_score(result_df)

#Convert the dataframe parameters into integers as they are returned as floats
lstm_test_df.essay_id = lstm_test_df.essay_id.astype(int)
lstm_test_df.essay_set = lstm_test_df.essay_set.astype(int)
lstm_test_df.Orig_Score = lstm_test_df.Orig_Score.astype(int)
lstm_test_df.Max_Score = lstm_test_df.Max_Score.astype(int)
lstm_test_df.Min_Score = lstm_test_df.Min_Score.astype(int)
lstm_test_df.Mean_Score = lstm_test_df.Mean_Score.astype(int)

# Extract the scores
orig_score = lstm_test_df.Orig_Score.values
max_pred_score = lstm_test_df.Max_Score.values
min_pred_score = lstm_test_df.Min_Score.values
mean_pred_score = lstm_test_df.Mean_Score.values

# Compare the metrics if Max score of all sentences are taken
rmse_max_lstm = RMSE(orig_score,max_pred_score)
cohen_kappa_max_lstm = cohen_kappa_score(orig_score,max_pred_score)
accuracy_max_lstm = accuracy_score(orig_score,max_pred_score)

# Compare the metrics if Min score of all sentences are taken
rmse_min_lstm = RMSE(orig_score,min_pred_score)
cohen_kappa_min_lstm = cohen_kappa_score(orig_score,min_pred_score)
accuracy_min_lstm = accuracy_score(orig_score,min_pred_score)

# Compare the metrics if Mean score of all sentences are taken
rmse_mean_lstm = RMSE(orig_score,mean_pred_score)
cohen_kappa_mean_lstm = cohen_kappa_score(orig_score,mean_pred_score)
accuracy_mean_lstm = accuracy_score(orig_score,mean_pred_score)

print("### Stacked LSTM Results : MAX Score of all Sentences ###")
print("RMSE: ",rmse_max_lstm)
print("Cohen Kappa: ",cohen_kappa_max_lstm)
print("Accuracy: ",accuracy_max_lstm)

print("### Stacked LSTM Results : Min Score of all Sentences ###")
print("RMSE: ",rmse_min_lstm)
print("Cohen Kappa: ",cohen_kappa_min_lstm)
print("Accuracy: ",accuracy_min_lstm)

print("### Stacked LSTM Results : Mean Score of all Sentences ###")
print("RMSE: ",rmse_mean_lstm)
print("Cohen Kappa: ",cohen_kappa_mean_lstm)
print("Accuracy: ",accuracy_mean_lstm)

# Provide per-essay-set metrics based on mean score
essay_set_results = essay_set_metrics(lstm_test_df)
essay_set_results

### Stacked LSTM Results : MAX Score of all Sentences ###
RMSE:  2.4402726615191552
Cohen Kappa:  0.26221593989910885
Accuracy:  0.3559322033898305
### Stacked LSTM Results : Min Score of all Sentences ###
RMSE:  2.5439889573463472
Cohen Kappa:  0.2768123741807381
Accuracy:  0.37211093990755006
### Stacked LSTM Results : Mean Score of all Sentences ###
RMSE:  1.9800624104963735
Cohen Kappa:  0.32899367937320956
Accuracy:  0.4179506933744222


,essay_set,RMSE,Kappa,Accuracy
0,1.0,1.384931,0.043037,0.281421
1,2.0,0.738549,0.283577,0.580214
2,3.0,0.767998,0.078550,0.410180
3,4.0,0.598609,0.494258,0.650000
4,5.0,0.825324,0.182558,0.457971
5,6.0,0.723747,0.211094,0.523810
6,7.0,3.926898,0.018414,0.099338
7,8.0,5.305279,0.007525,0.058394


#### RNN GRU

In [34]:
# Train the model
estimator = KerasRegressor(build_fn=GRU, epochs=20, batch_size=500)
estimator.fit(train_data, y_train)

# Predict for test data
prediction_rnn=estimator.predict(test_data)
rmse_val = RMSE(y_test,prediction_rnn)
# Find the overall RMSE value. This is not very relevant considering different scale for each set.
print("RMSE: ",rmse_val)

# Construct pandas data frame of scores of each sentences
rnn_df = pd.DataFrame([x_test_sentence_df['domain1_score'].astype(np.double), y_test.astype(np.double),
                      prediction_rnn.astype(np.double)]).transpose()
rnn_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
# Find Multiplication factor using the function we defined before
rnn_df['Mult_Factor'] = rnn_df.apply(find_mult_factor,axis=1)
# Find the denormalized predicted score
rnn_df['Denorm_Pred_Score'] = rnn_df.apply(denormalize,axis=1)

# Extract the scores of each sentences
orig_score = rnn_df.Orig_Score.as_matrix(columns=None)
orig_score = orig_score.astype(np.int)
pred_score = rnn_df.Denorm_Pred_Score.as_matrix(columns=None)
pred_score = pred_score.astype(np.int)

# Provide the metrics at sentence level
rmse_rnn = RMSE(orig_score,pred_score)
cohen_kappa = cohen_kappa_score(orig_score,pred_score)
accuracy = accuracy_score(orig_score,pred_score)
print("RMSE Sentence Level: ",rmse_rnn)
print("Kappa Sentence Level: ",cohen_kappa)
print("Accuracy Sentence Level: ",accuracy)




Epoch 1/20
147123/147123 [==============================] - 88s 599us/step - loss: 0.0714 - acc: 0.0879
Epoch 2/20
147123/147123 [==============================] - 88s 596us/step - loss: 0.0357 - acc: 0.0979
Epoch 3/20
147123/147123 [==============================] - 87s 589us/step - loss: 0.0311 - acc: 0.0993
Epoch 4/20
147123/147123 [==============================] - 87s 588us/step - loss: 0.0293 - acc: 0.1000
Epoch 5/20
147123/147123 [==============================] - 86s 584us/step - loss: 0.0283 - acc: 0.1004
Epoch 6/20
147123/147123 [==============================] - 86s 584us/step - loss: 0.0276 - acc: 0.1005
Epoch 7/20
147123/147123 [==============================] - 86s 585us/step - loss: 0.0271 - acc: 0.1008
Epoch 8/20
147123/147123 [==============================] - 86s 588us/step - loss: 0.0268 - acc: 0.1009
Epoch 9/20
147123/147123 [==============================] - 87s 588us/step - loss: 0.0263 - acc: 0.1012
Epoch 10/20
147123/147123 [==============================] - 87s

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [35]:
# We will combine the sentences back to essay in this section

result_df = pd.DataFrame([x_test_sentence_df.essay_id.values,x_test_sentence_df.essay_set.values,
                         x_test_sentence_df.sentence.values,rnn_df.Orig_Score.values,
                          rnn_df.Denorm_Pred_Score.values]).transpose()
result_df.columns = ['essay_id','essay_set','sentence','orig_score','pred_score']
rnn_test_df = find_max_min_mean_score(result_df)

#Convert the dataframe parameters into integers as they are returned as floats
rnn_test_df.essay_id = rnn_test_df.essay_id.astype(int)
rnn_test_df.essay_set = rnn_test_df.essay_set.astype(int)
rnn_test_df.Orig_Score = rnn_test_df.Orig_Score.astype(int)
rnn_test_df.Max_Score = rnn_test_df.Max_Score.astype(int)
rnn_test_df.Min_Score = rnn_test_df.Min_Score.astype(int)
rnn_test_df.Mean_Score = rnn_test_df.Mean_Score.astype(int)

# Extract the scores
orig_score = rnn_test_df.Orig_Score.values
max_pred_score = rnn_test_df.Max_Score.values
min_pred_score = rnn_test_df.Min_Score.values
mean_pred_score = rnn_test_df.Mean_Score.values

# Compare the metrics if Max score of all sentences are taken
rmse_max_rnn = RMSE(orig_score,max_pred_score)
cohen_kappa_max_rnn = cohen_kappa_score(orig_score,max_pred_score)
accuracy_max_rnn = accuracy_score(orig_score,max_pred_score)

# Compare the metrics if Min score of all sentences are taken
rmse_min_rnn = RMSE(orig_score,min_pred_score)
cohen_kappa_min_rnn = cohen_kappa_score(orig_score,min_pred_score)
accuracy_min_rnn = accuracy_score(orig_score,min_pred_score)

# Compare the metrics if Mean score of all sentences are taken
rmse_mean_rnn = RMSE(orig_score,mean_pred_score)
cohen_kappa_mean_rnn = cohen_kappa_score(orig_score,mean_pred_score)
accuracy_mean_rnn = accuracy_score(orig_score,mean_pred_score)

print("### RNN GRU Results : MAX Score of all Sentences ###")
print("RMSE: ",rmse_max_rnn)
print("Cohen Kappa: ",cohen_kappa_max_rnn)
print("Accuracy: ",accuracy_max_rnn)

print("### RNN GRU Results : Min Score of all Sentences ###")
print("RMSE: ",rmse_min_rnn)
print("Cohen Kappa: ",cohen_kappa_min_rnn)
print("Accuracy: ",accuracy_min_rnn)

print("### RNN GRU Results : Mean Score of all Sentences ###")
print("RMSE: ",rmse_mean_rnn)
print("Cohen Kappa: ",cohen_kappa_mean_rnn)
print("Accuracy: ",accuracy_mean_rnn)

# Provide per-essay-set metrics based on mean score
essay_set_results = essay_set_metrics(rnn_test_df)
essay_set_results

### RNN GRU Results : MAX Score of all Sentences ###
RMSE:  2.47802359240166
Cohen Kappa:  0.24640110969689533
Accuracy:  0.33782742681047767
### RNN GRU Results : Min Score of all Sentences ###
RMSE:  2.48988709591122
Cohen Kappa:  0.2994161154737035
Accuracy:  0.3940677966101695
### RNN GRU Results : Mean Score of all Sentences ###
RMSE:  1.966299270132258
Cohen Kappa:  0.3049514854524614
Accuracy:  0.3956086286594761


,essay_set,RMSE,Kappa,Accuracy
0,1.0,1.426716,0.028712,0.229508
1,2.0,0.775632,0.237609,0.548128
2,3.0,0.777683,0.056194,0.395210
3,4.0,0.598609,0.483702,0.641667
4,5.0,0.827078,0.153610,0.437681
5,6.0,0.768080,0.164754,0.505291
6,7.0,3.863996,0.000635,0.086093
7,8.0,5.244392,-0.002928,0.051095


#### Feed Forward NN

In [36]:
# Train the model
estimator_nn = KerasRegressor(build_fn=FF_NN, epochs=20, batch_size=500)
estimator_nn.fit(train_data, y_train)

# Predict for test data
prediction_nn=estimator_nn.predict(test_data)
rmse_val = RMSE(y_test,prediction_nn)
# Find the overall RMSE value. This is not very relevant considering different scale for each set.
print("RMSE: ",rmse_val)

# Construct pandas data frame of scores of each sentences
nn_df = pd.DataFrame([x_test_sentence_df['domain1_score'].astype(np.double), y_test.astype(np.double),
                      prediction_nn.astype(np.double)]).transpose()
nn_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
# Find Multiplication factor using the function we defined before
nn_df['Mult_Factor'] = nn_df.apply(find_mult_factor,axis=1)
# Find the denormalized predicted score
nn_df['Denorm_Pred_Score'] = nn_df.apply(denormalize,axis=1)

# Extract the scores of each sentences
orig_score = nn_df.Orig_Score.as_matrix(columns=None)
orig_score = orig_score.astype(np.int)
pred_score = nn_df.Denorm_Pred_Score.as_matrix(columns=None)
pred_score = pred_score.astype(np.int)

# Provide the metrics at sentence level
rmse_nn = RMSE(orig_score,pred_score)
cohen_kappa = cohen_kappa_score(orig_score,pred_score)
accuracy = accuracy_score(orig_score,pred_score)
print("RMSE Sentence Level: ",rmse_nn)
print("Kappa Sentence Level: ",cohen_kappa)
print("Accuracy Sentence Level: ",accuracy)

Epoch 1/20
147123/147123 [==============================] - 3s 18us/step - loss: 0.0451 - acc: 0.0983
Epoch 2/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0291 - acc: 0.0990
Epoch 3/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0278 - acc: 0.0994
Epoch 4/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0268 - acc: 0.1001
Epoch 5/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0259 - acc: 0.1005
Epoch 6/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0251 - acc: 0.1011
Epoch 7/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0242 - acc: 0.1015
Epoch 8/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0234 - acc: 0.1020
Epoch 9/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0224 - acc: 0.1025
Epoch 10/20
147123/147123 [==============================] - 1s 7us/step - loss: 0.0216 - 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [37]:
# We will combine the sentences back to essay in this section

result_df = pd.DataFrame([x_test_sentence_df.essay_id.values,x_test_sentence_df.essay_set.values,
                         x_test_sentence_df.sentence.values,nn_df.Orig_Score.values,
                          nn_df.Denorm_Pred_Score.values]).transpose()
result_df.columns = ['essay_id','essay_set','sentence','orig_score','pred_score']
nn_test_df = find_max_min_mean_score(result_df)

#Convert the dataframe parameters into integers as they are returned as floats
nn_test_df.essay_id = nn_test_df.essay_id.astype(int)
nn_test_df.essay_set = nn_test_df.essay_set.astype(int)
nn_test_df.Orig_Score = nn_test_df.Orig_Score.astype(int)
nn_test_df.Max_Score = nn_test_df.Max_Score.astype(int)
nn_test_df.Min_Score = nn_test_df.Min_Score.astype(int)
nn_test_df.Mean_Score = nn_test_df.Mean_Score.astype(int)

# Extract the scores
orig_score = nn_test_df.Orig_Score.values
max_pred_score = nn_test_df.Max_Score.values
min_pred_score = nn_test_df.Min_Score.values
mean_pred_score = nn_test_df.Mean_Score.values

# Compare the metrics if Max score of all sentences are taken
rmse_max_nn = RMSE(orig_score,max_pred_score)
cohen_kappa_max_nn = cohen_kappa_score(orig_score,max_pred_score)
accuracy_max_nn = accuracy_score(orig_score,max_pred_score)

# Compare the metrics if Min score of all sentences are taken
rmse_min_nn = RMSE(orig_score,min_pred_score)
cohen_kappa_min_nn = cohen_kappa_score(orig_score,min_pred_score)
accuracy_min_nn = accuracy_score(orig_score,min_pred_score)

# Compare the metrics if Mean score of all sentences are taken
rmse_mean_nn = RMSE(orig_score,mean_pred_score)
cohen_kappa_mean_nn = cohen_kappa_score(orig_score,mean_pred_score)
accuracy_mean_nn = accuracy_score(orig_score,mean_pred_score)

print("### Simple FF NN Results : MAX Score of all Sentences ###")
print("RMSE: ",rmse_max_nn)
print("Cohen Kappa: ",cohen_kappa_max_nn)
print("Accuracy: ",accuracy_max_nn)

print("### Simple FF NN Results : Min Score of all Sentences ###")
print("RMSE: ",rmse_min_nn)
print("Cohen Kappa: ",cohen_kappa_min_nn)
print("Accuracy: ",accuracy_min_nn)

print("### Simple FF NN Results : Mean Score of all Sentences ###")
print("RMSE: ",rmse_mean_nn)
print("Cohen Kappa: ",cohen_kappa_mean_nn)
print("Accuracy: ",accuracy_mean_nn)

# Provide per-essay-set metrics based on mean score
essay_set_results = essay_set_metrics(nn_test_df)
essay_set_results

### Simple FF NN Results : MAX Score of all Sentences ###
RMSE:  3.132721585147047
Cohen Kappa:  0.1953363332195014
Accuracy:  0.288135593220339
### Simple FF NN Results : Min Score of all Sentences ###
RMSE:  3.3338854190978644
Cohen Kappa:  0.21383702599770982
Accuracy:  0.31587057010785824
### Simple FF NN Results : Mean Score of all Sentences ###
RMSE:  2.071153404571869
Cohen Kappa:  0.2887425982170171
Accuracy:  0.3817411402157165


,essay_set,RMSE,Kappa,Accuracy
0,1.0,1.442902,0.035802,0.240437
1,2.0,0.802736,0.203135,0.526738
2,3.0,0.789148,0.056629,0.395210
3,4.0,0.654047,0.383315,0.572222
4,5.0,0.879723,0.138566,0.426087
5,6.0,0.835711,0.141745,0.500000
6,7.0,4.221515,0.004166,0.082781
7,8.0,5.306655,0.003413,0.051095
